In [ ]:
#From Jesse's question_generation.ipynb
from nltk.tree import Tree

def list_to_string(word_list):
    return ' '.join(word_list)

def tree_to_string(parsed_tree):
#     if isinstance(parsed_tree, str):
#         return parsed_tree
#     words = []
#     for subtree in parsed_tree:
#         words.append(tree_to_string(subtree))
    return list_to_string(parsed_tree.leaves())

def binary_question_from_tree(parsed_tree):
    sentence = parsed_tree[0]
    assert(sentence.label() == 'S')
    np = sentence[0]
    vp = sentence[1]
    assert(np.label() == 'NP')
    assert(vp.label() == 'VP')
    if vp[0].label() == 'VBZ':
        return list_to_string([vp[0][0].capitalize(), tree_to_string(np), tree_to_string(vp[1])]) + '?'
    return vp[0]

#Sentence Structure Tree
class SST():
    def __init__(self, label, children):
        self.label = label
        self.children = children

#Sentence Structure Leaf
class SSL():
    def __init__(self, label):
        self.label = label
        
simple_predicate = SST('ROOT', [SST('S', [SSL('NP'), SSL('VP'), SSL('.')])])

def satisfies_structure(parsed_tree, structure):
    if isinstance(structure, SSL):
        return parsed_tree.label() == structure.label
    else:
        if parsed_tree.label() != structure.label or len(parsed_tree) != len(structure.children): return False
        for i in range(len(parsed_tree)):
            if satisfies_structure(parsed_tree[i], structure.children[i]) == False:
                return False
        return True

In [ ]:
#From Jesse's question_generation.ipynb pt2
parser = CoreNLPParser(url='http://localhost:9000')
parse_list = []
sentences = ["Gyarados is voiced by Unshō Ishizuka in both Japanese and English media."]
for sentence in sentences:
    if len(sentence) < 180:
        parse = next(parser.raw_parse(sentence))
        if satisfies_structure(parse, simple_predicate):
            print("=========================== Sentence ======================")
            print(parse)
#             print(parse.label())
            print(sentence) 
            print(binary_question_from_tree(parse))
            parse_list.append(parse)
            
parse.draw()